## Data Cleaning
We will be cleaning the data to be eventually used to train the model to predict influenza cases.

We start off with importing all the libraries we are going to use for this.

In [65]:
import pandas as pd
from pathlib import Path
from config import COUNTRIES

We set up our data paths. We had already combined the data from the several different csv files into one a single one using the `combine.py` file in the `src` module.

These files are named after their countries and present in the data/combined directory.

In [66]:
path = Path.cwd()
data_path = path.parent / 'data' / 'combined'

### Making Dataframes from the .csv files

`df`, here, contrary to conventional use, doesn't refer to the name of a pandas dataframe. Instead it is a dictionary of pandas dataframes. The key is the country name and the value is the dataframe corresponding to that country.

In [67]:
df = {}
for country in COUNTRIES :
    file_path = data_path / (country+'.csv')
    df[country] = pd.read_csv(file_path)

### Cleaning data for Austria
We will start off with cleaning the data for just one country. If you notice, all the data in all the files is pretty similar. It is after all the same data, but for different countries. What we deduce from the dataframe for *austria*, we shall implement for all other countries.

Just to get an idea about what is this data we are  dealing with:

In [68]:
df['austria'].describe()

,"""RTS,S""",Acetaldehydsyndrom,Achsensymptom,Adeno-assoziierte_Viren,Adenovirusimpfstoff,Adipsie,Adynamie,Aggravation,Agitation_(Medizin),Akantholyse,...,Zervizitis,Zikavirus-Epidemie_2015/2016,Zohlen-Zeichen,Zoonose,Zwerchfellhochstand,Zyanose,cases,incidence,Ödem,Übelkeit
count,156.0,156.000000,156.000000,156.000000,156.000000,156.000000,156.000000,144.000000,156.0,156.000000,...,156.000000,156.000000,156.000000,66.000000,156.000000,156.000000,130.000000,156.000000,156.000000,156.000000
mean,0.0,106.044872,2.307692,114.685897,4.512821,107.423077,242.121795,404.152778,0.0,111.160256,...,23.044872,21.217949,185.237179,1330.560606,262.717949,1933.211538,398.892308,880.461157,1579.358974,251.942308
std,0.0,35.336419,2.699141,29.542709,5.373156,62.227578,96.730777,82.270177,0.0,43.810663,...,33.477646,32.972578,85.005891,360.611068,112.797781,588.776446,239.507934,429.093467,2262.157075,360.486486
min,0.0,48.000000,0.000000,31.000000,0.000000,26.000000,79.000000,140.000000,0.0,33.000000,...,0.000000,0.000000,45.000000,587.000000,95.000000,691.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,81.750000,0.000000,97.000000,0.000000,60.000000,173.000000,365.250000,0.0,85.000000,...,0.000000,0.000000,127.750000,1074.250000,180.500000,1461.250000,258.000000,636.664425,0.000000,0.000000
50%,0.0,96.000000,2.000000,111.000000,1.000000,80.000000,216.500000,399.500000,0.0,99.000000,...,1.000000,0.000000,158.000000,1356.000000,220.000000,1871.500000,380.000000,816.951024,7.000000,3.000000
75%,0.0,130.250000,3.000000,132.000000,8.000000,164.500000,289.750000,450.000000,0.0,131.000000,...,55.250000,45.750000,230.000000,1508.750000,345.750000,2342.250000,532.000000,1120.661459,4296.250000,665.250000
max,0.0,216.000000,12.000000,193.000000,21.000000,264.000000,511.000000,651.000000,0.0,334.000000,...,115.000000,150.000000,492.000000,2980.000000,568.000000,3600.000000,992.000000,2052.819744,5756.000000,1004.000000


In [69]:
print(df['austria'].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Columns: 385 entries, "RTS,S" to date
dtypes: float64(383), object(2)
memory usage: 469.3+ KB
None


#### Dealing with missing values

The first instinct when dealing with missing data is to try and see if there is any easy way out. We will start of by dropping all rows where any element is zero:

In [70]:
df['austria'].dropna().shape

(0, 385)

As we can see, doing this reduced our dataset to zero rows. Every row has some or the other null value.

We can now try a column specific approaach. i.e. dropping all columns with even one single null value:

In [71]:
df['austria'].dropna(axis=1).shape

(156, 311)

The number of columns reduced from 383 to 311 , that is a little over 80%. This is a good enough number of non-null features.

However, lets try to see if we can increase this number. We shall proceed to check how many values are null in each of those 72 columns:

In [72]:
total = df['austria'].isnull().sum().sort_values(ascending=False)
total.head()

Otitis_externa    156
Syndrom           156
Epidemie          155
VSV-EBOV          154
Bartholinitis     150
dtype: int64

You know what would be more helpfull that this? If we could get a percentage estimation of this. as in, instead of how many rows have null, how much percentage of rows have null values.

In [73]:
def find_null_percent(data):
    percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
    return percent

In [74]:
find_null_percent(df['austria']).head()

Otitis_externa    1.000000
Syndrom           1.000000
Epidemie          0.993590
VSV-EBOV          0.987179
Bartholinitis     0.961538
dtype: float64

As far as those column are concerned where the amount of data missing is more than 20%(0.2), if we try to infer the data, it will most probably be wrong and can misguide our training model.

We drop those columns from our dataset where the number of null values is more than 20%

In [75]:
null_percent_list = find_null_percent(df['austria'])
for column, null_percent in null_percent_list.items() :
    if null_percent > 0.2:
        df['austria'] = df['austria'].drop(columns=[column])

Let's see what the damage is:

In [76]:
df['austria'].shape

(156, 316)

In [77]:
find_null_percent(df['austria']).head(10)

Parotitis                 0.192308
cases                     0.166667
Rhinosinusitis            0.153846
Fieber                    0.108974
Aggravation               0.076923
date                      0.000000
Hospitalfieber            0.000000
Herdenzephalitis          0.000000
Hexavalenter_Impfstoff    0.000000
Hexenschuss               0.000000
dtype: float64

Only 5 columns having any null values have null values less than 20%. Let's see if we can try to infer this data from the rest of the data.

Also, the columns called *cases* isn't really a list of wikipedia pageviews for any article. So we get rid of that.

In [78]:
df['austria'] = df['austria'].drop(columns='cases')


Now we have to deal with these 4 columns.
Since this data is similar to a time series, we can use methods used to there to infer the mssing data.

Namely linear interpollation.

In [79]:
df['austria'] = df['austria'].interpolate(method='linear', limit_direction='forward', axis=0)
df['austria'] = df['austria'].interpolate(method='linear', limit_direction='backward', axis=0)

We have successfully gotten rid of all null values in this dataframe. Let's verify this:

In [80]:
find_null_percent(df['austria']).head()

date                      0.0
Hypalbuminämie            0.0
Herdenzephalitis          0.0
Hexavalenter_Impfstoff    0.0
Hexenschuss               0.0
dtype: float64

Now we have to do what we have done here to all the countries.

We will do this via a simple for loop.

In [81]:
for country in COUNTRIES:
        if country == 'austria':
            continue
        print('For ' + country)
        print('\tnumber of columns before cleaning is '+ str(df[country].shape[1]))

        if 'cases' in df[country].columns:
            df_country = df[country].drop(columns=['cases'])

        # drop columns with all more than 20% data missing
        percent_missing = find_null_percent(df['austria'])

        for column, null_percent in percent_missing.items():
            if null_percent > 0.2:
                df[country] = df[country].drop(columns=[column])

        # perform linear interpolation on remaining missing values
        df[country] = df[country].interpolate(method='linear', limit_direction='forward', axis=0)
        df[country] = df[country].interpolate(method='linear', limit_direction='backward', axis=0)

        # drop columns with all values as zeroes
        df[country] = df[country].loc[:, (df[country] != 0).any(axis=0)]

        print('\tnumber of columns after cleaning is '+ str(df[country].shape[1]))

For belgium
	number of columns before cleaning is 385
	number of columns after cleaning is 367
For germany
	number of columns before cleaning is 384
	number of columns after cleaning is 372
For italy
	number of columns before cleaning is 473
	number of columns after cleaning is 473
For netherlands
	number of columns before cleaning is 385
	number of columns after cleaning is 367


We have successfully cleaned all country dataframes.